In [ ]:
import numpy as np
import json
import os,sys
import pandas as pd
import operator
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
import datetime


# train_dir_path='/home/irlplab/Documents/Share/Akansha/GoogleNLQ/natural_questions/v1.0/train/'
# test_dir_path='/home/irlplab/Documents/Share/Akansha/GoogleNLQ/natural_questions/v1.0/dev/'

test_dir_path='/home/irlplab/Documents/Share/Akansha/GoogleNLQ/'
# test_dir_path='/home/irlplab/Documents/Share/Akansha/temp/test/'
train_dir_path='/home/irlplab/Documents/Share/Akansha/temp/train/'

test_data=[]

train_data = {}

test_url = set()

null_obj={
        "candidate_index": -1,
        "end_byte": -1,
        "end_token": -1,
        "start_byte": -1,
        "start_token": -1
      }


beta = 3
true_pos=0
true_neg=0
recall_den=0
precision_den=0
matched_doc=0

no_of_annotators = 5

def load_json_multiple(segments):
    chunk = ""
    for segment in segments:
        
        chunk += segment
        try:
            yield json.loads(chunk)
            chunk = ""
        except ValueError:
            pass


def get_test_data():
    json_files = [pos_json for pos_json in os.listdir(test_dir_path) if pos_json.endswith('.jsonl')]
    all_data=[]
#     print(json_files)
    global test_url
    for index, js in enumerate(json_files):
        with open(os.path.join(test_dir_path, js)) as json_file:
            for parsed_json in load_json_multiple(json_file):
                
                data = {}
                data["example_id"]=parsed_json['example_id']
                data["question_text"]=parsed_json['question_text']
                data["annotations"]=parsed_json['annotations']
                data["document_url"]=parsed_json['document_url']
                test_url.add(parsed_json['document_url'])
                all_data.append(data)
                
    print("test data length:"+str(len(all_data)))
    print("Total urls in test:"+str(len(test_url)))
    return all_data

def get_annotation_text(ex_obj):
    start_token = ex_obj['annotations'][0]['long_answer']['start_token']
    end_token = ex_obj['annotations'][0]['long_answer']['end_token']
#     print(type(start_token))
    text = ""
    while(start_token <= end_token):
        if(ex_obj['document_tokens'][start_token]['html_token'] == False):
            text = text+" "+ex_obj['document_tokens'][start_token]['token']
        start_token = start_token+1
    return text
    


def get_train_data(segments):
    
    for segment in segments:
        global train_data
        ex_obj = json.loads(segment)
        if(ex_obj['document_url'] in test_url):
            
            if ex_obj['document_url'] not in train_data:
                train_data[ex_obj['document_url']]=[]
            
            data = {}
            data["example_id"]=ex_obj['example_id']
            data["annotations"]=ex_obj['annotations'][0]['long_answer']
            data["text"]= get_annotation_text(ex_obj)
#             print("train text::"+str(data))
            if data["text"]:
                
                train_data[ex_obj['document_url']].append(data)
            


def create_url_dict():
    json_files = [pos_json for pos_json in os.listdir(train_dir_path) if pos_json.endswith('.jsonl')]
    for index, js in enumerate(json_files):
        with open(os.path.join(train_dir_path, js)) as json_file:
            get_train_data(json_file)

            
def get_closest_document_index(train_vector,query_vector):
    
    cosine_similarities = linear_kernel(query_vector, train_vector).flatten()
    index, value = max(enumerate(cosine_similarities), key=operator.itemgetter(1))
    return index

    
def get_long_ans():
    global null_obj
    global test_data
    global matched_doc
#     print("train data "+str(train_data))
    for idx,test_que in enumerate(test_data):
        query=test_que['question_text']
        url=test_que['document_url']
        if url in train_data:
            matched_doc = matched_doc+1
            answer_obj=train_data[url]
#             print("size is "+str(len(answer_obj)))
            all_text=[]
            print(answer_obj)
            for i in range(len(answer_obj)):
                all_text.append(answer_obj[i]['text'])
            print(all_text)
            if(len(all_text)>0):
                vectorizer = TfidfVectorizer(stop_words=None)
                train_vector = vectorizer.fit_transform(all_text)
                query_vector = vectorizer.transform([query])
                closest_index=get_closest_document_index(train_vector,query_vector)
                h_function(answer_obj[closest_index]['annotations'],test_que['annotations'])
                    
            else:
                h_function(null_obj,test_que['annotations'])        
        else:
            h_function(null_obj,test_que['annotations'])
        
    print (matched_doc)
    
    
def h_function(predicted_answer,annotator_answer):
    
#     print("Predicted ans : " + str(predicted_answer))
#     print("Annotated ans : " + str(annotator_answer))
    
    match_flag = 0
    g_count = 0
    global true_pos
    global true_neg
    global recall_den
    global precision_den

    for i in range(no_of_annotators):
        if(annotator_answer[i]['long_answer']['candidate_index'] != -1):
            g_count = g_count+1
    
    if(g_count>=beta):
        recall_den=recall_den+1
    
    if(predicted_answer['candidate_index']!=-1):
        precision_den=precision_den+1
    
    if(predicted_answer['candidate_index']!=-1 and g_count >=beta):
        for i in range(no_of_annotators):               
            if(annotator_answer[i]['long_answer']['candidate_index'] == predicted_answer['candidate_index'] and \
               annotator_answer[i]['long_answer']['end_byte'] == predicted_answer['end_byte'] and \
               annotator_answer[i]['long_answer']['end_token'] == predicted_answer['end_token'] and \
               annotator_answer[i]['long_answer']['start_byte'] == predicted_answer['start_byte'] and\
               annotator_answer[i]['long_answer']['start_token'] == predicted_answer['start_token']):
                match_flag=1
                true_pos=true_pos+1
                print("first!!")
                return 1
        if(match_flag==0):
            print("second!!")
            return 0
            
            
    elif(g_count < beta and predicted_answer['candidate_index']==-1):
        true_neg=true_neg+1
        print("third!!")
        print("Predicted ans : " + str(predicted_answer))
        print("Annotated ans : " + str(annotator_answer))
        return 1
    
    else:
        print("fourth!!")
        return 0
        
def f1_score():
    recall=true_pos/recall_den
    
    print("recall "+ str(recall))
    
    precision=true_pos/precision_den
    
    print("Precision "+ str(precision))
    
    if(precision!=0 and recall!=0):
        f1=2*(precision*recall)/(precision+recall)
    else:
        f1=0
    print("F1 Score "+ str(f1))
       
    print("true_pos "+str(true_pos))
    print("true_neg "+str(true_neg))
    print("recall_den "+str(recall_den))
    print("precision_den "+str(precision_den))

        
def main():
       
    print ("Start time")
    currentDT1 = datetime.datetime.now()
    print (str(currentDT1))

    global test_data
    test_data=get_test_data()
    
    global train_data
    create_url_dict()
    
    get_long_ans()
    f1_score()

    print ("End time")
    currentDT2 = datetime.datetime.now()
    print (str(currentDT2))

if __name__ == '__main__':
    main()



    
    


Start time
2019-02-28 14:13:18.260600
test data length:3294
Total urls in test:3003
